# Gaussian Elimination with Pivoting

In [1]:
import numpy as np

### Gaussian Elimination with Partial Pivoting

- In place of choosing the default row (row $k$) on the $k$-th pass, the  <b>pivot row</b> is chosen to have largest absolute value element ( pivot element) in the $k$-th column below and including row $k$.
 
- Row interchange takes place to make the row having the pivot element as the pivot row. (Not really in the algorithm though. A permutation vector, $p$, can keep track of the row interchanges instead. Here $p$ is initialized as $p = [1,\, 2,\, 3, \cdots,\, n]$.)
 
- The Gaussian elimination could be used for LU-decomposition  of matrix $PA$ where permutation matrix $P$ is associated with the permutation vector $p$
$$LU = PA.$$


### Gaussian elimination with scaled-row partial pivoting
---
<b>Scaled Row Partial Pivoting</b> (SRPP) is a pivoting strategy in which row-interchanges are based on the scaling of potential pivot elements by the absolute maximum element of each row.


- In Gaussian elimination with SRPP, we use a scaling vector
$s = [s_1, s_2, \cdots, s_n]$,
where
$$s_i = \max_{1  \leq j \leq n} |a_{ij}| , \qquad (1  \leq i \leq n)$$

- The pivot element (and pivot-row) at pass-$k$ is chosen based on the maximum of 
$$
\max \left \{
\frac{|a_{p_i k}|}{s_{p_i}} ;  \qquad k \leq i \leq n .
\right \}
$$
where $p$ is a permuations vectors that keeps track of row interchanges. It is initialized with default ordering $[1,\ 2,\ 3, \cdots \ n ]$. Rows swaps are logically performed by intercahnging the elements pf $p$. For example if row 1 is changed with row 2, then $p$ changes to $p=[2,\ 1,\ 3, \ \cdots \ n ]$.
 
- The Gaussian elimination could again be used for LU-decomposition  as 
$$LU = PA,$$
where is $P$ is the permutation matrix associated with vector $p$, given by 
$$
P=I_n[p,\,:]
$$
For example, if $p=[3,1,2]$, the corresponding permutation matrix is given by

$$
P=I_3[p,\,:] =I_3[[3,1,2],\,:] =  \begin{bmatrix}
0 & 0 & 1  \\
1 & 0 & 0  \\
0 & 1 & 0 
\end{bmatrix}
$$

The following code provides an implementation of Gaussian Elimination with scaled row-partial pivoting. Ponder over the following.

  - What is the need for pivoting?
  - How can you find a factorization of the matrix by using the following code.
  - How can you modify the code to find the determinant of a matrix?
  - How can you modify the following code to solve a linear system?
  - How can you modify the code to find the inverse of a matrix?

In [3]:
## Gaussian Elimination: with Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_srpp(X, verbose=False):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    A = np.copy(X)
    m,n = A.shape
    swap=0;
    
    # The initial ordering of rows
    p = list(range(m))
    if verbose:
        print("permutation vector initialized to: ",p)
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1) 
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(m-1):              
        if verbose:
            print("Scaling Vector: ",s)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange elements in the permutation vector if needed
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            swap+=1;
                        
        if verbose:
            print("\nPivot Element: {0:.4f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular or there is a zero in pivot position.")        
        # Check the new order of rows
        if verbose:
            print("permutation vector: ",p)
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, m):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]            
            #Save the multiplier z in A itself. You can save this in L also
            A[p[i],k] = z          
            #Elimination operation: Changes all elements in a row simultaneously
            A[p[i],k+1:] = A[p[i],k+1:] - z*A[p[k],k+1:]
            
        if verbose:
            #print("\n After PASS {}=========: \n".format(k+1), A)
            print("\n After PASS {}=========: \n".format(k+1), bmatrix(A))
            
    return A, p, swap

In [10]:
# In the following function A is supposed to be upper triangular systems UX=b
def back_sub(U, b):
    n = U.shape[0]
    #Solution will be saved in variable x
    x = np.zeros_like(b, dtype=float)

    # Back-substitution
    ##last variable is found first
    x[n-1] = b[n-1] / U[n-1,n-1]
    ## Find the remaining n-1 variables from last to first
    for k in range(n-2,-1,-1):
        known_sums = np.dot(U[k,k+1:],x[k+1:])
        x[k] = (b[k] - known_sums) / U[k,k]
    return x

In [4]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
A0 = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
#A = np.array([[5, 4, 7, 6, 9], [7, 8, 9, 9, 8], [2, 3, 5, 9, 8], [3, 1, 7, 5, 6], [9, 1, 3, 7, 3]], dtype=float)
#A0=np.array([[1, 0, 2, 1],[4, -9, 2, 1],[8, 16, 6, 5],[2, 3, 2,1]], dtype=float)
print("\n Given A: \n ",bmatrix(A0))


 Given A: 
  \begin{bmatrix}
  2. & 3. & -6.\\
  1. & -6. & 8.\\
  3. & -2. & 1.\\
\end{bmatrix}


In [5]:
A,p,swap =GE_srpp(A0,verbose=True) #You need to pass a copy, otherwise the original A0 is changed.
m,n=A.shape

permutation vector initialized to:  [0, 1, 2]
Scaling Vector:  [6. 8. 3.]

Pivot Element: 3.0000 

permutation vector:  [2, 1, 0]

 After PASS 1=========: 
 \begin{bmatrix}
  0.66666667 & 4.33333333 & -6.66666667\\
  0.33333333 & -5.33333333 & 7.66666667\\
  3. & -2. & 1.\\
\end{bmatrix}
Scaling Vector:  [6. 8. 3.]

Pivot Element: 4.3333 

permutation vector:  [2, 0, 1]

 After PASS 2=========: 
 \begin{bmatrix}
  0.66666667 & 4.33333333 & -6.66666667\\
  0.33333333 & -1.23076923 & -0.53846154\\
  3. & -2. & 1.\\
\end{bmatrix}


Here is a visualization for the above Gaussian elimination process with scaled row partial pivoting.

$$
 \begin{bmatrix}
  2. & 3. & -6.\\
  1. & -6. & 8.\\
  \color{red}{[3]}. & -2. & 1.\\
\end{bmatrix} \longrightarrow 
\begin{bmatrix}
  \color{blue}{(0.667)} & \color{red}{[4.333]} & -6.667\\
  \color{blue}{(0.333)} & -5.333 & 7.667\\
  \color{red}{[3]}. & -2. & 1.\\
\end{bmatrix}\longrightarrow 
\begin{bmatrix}
  \color{blue}{(0.667)} & \color{red}{[4.333]} & -6.667\\
  \color{blue}{(0.333)} & \color{blue}{(-1.231)} & -0.538\\
  \color{red}{[3]}. & -2. & 1.\\
\end{bmatrix}
$$

Note: The numbers of the form $\color{red}{[3]}$ are pivot elements.
The numbers of the form $\color{blue}{(0.667)}$ are zeros that are now storing the multipliers from the Gaussian Elimination.


In [7]:
L = np.tril(A[p,:], -1)+np.eye(n)
U = np.triu(A[p,:]) # YOu can solve Ux = b[p] by back substitution
P = np.eye(n)[p,:]
print("\n After Gaussian Elimination with SRPP: \n", A)
print("\n The permutation Vector is: \n", p)
print("\n Permutation matrix, P:\n ", P)
print("\n Upper triangular, U:\n ", U)
print("\n Lower triangular, L:\n", L)
print("Sanity Check: norm of PA-LU",np.linalg.norm(P@A0-L@U))


 After Gaussian Elimination with SRPP: 
 [[ 0.66666667  4.33333333 -6.66666667]
 [ 0.33333333 -1.23076923 -0.53846154]
 [ 3.         -2.          1.        ]]

 The permutation Vector is: 
 [2, 0, 1]

 Permutation matrix, P:
  [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

 Upper triangular, U:
  [[ 3.         -2.          1.        ]
 [ 0.          4.33333333 -6.66666667]
 [ 0.          0.         -0.53846154]]

 Lower triangular, L:
 [[ 1.          0.          0.        ]
 [ 0.66666667  1.          0.        ]
 [ 0.33333333 -1.23076923  1.        ]]
Sanity Check: norm of PA-LU 0.0


In [8]:
A0
print(A0)
b=np.array([-1,3,2], dtype=float).reshape(3,-1)
augA = np.hstack((A0,b))
print(augA)
print(np.hstack((A0,np.eye(3))))

[[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]
[[ 2.  3. -6. -1.]
 [ 1. -6.  8.  3.]
 [ 3. -2.  1.  2.]]
[[ 2.  3. -6.  1.  0.  0.]
 [ 1. -6.  8.  0.  1.  0.]
 [ 3. -2.  1.  0.  0.  1.]]


In [11]:
newA,p,swap = GE_srpp(augA)
newU = np.triu(newA[p,:3])
newb = newA[p,3]
print(back_sub(newU, newb))

[1. 1. 1.]


In [12]:
A_test = np.array([[3, -5, 1], 
              [1, 2, 3], 
              [-2, 3, -4]], dtype=float)
b = np.array([[0],[1],[3]], dtype=float)
augA = np.hstack((A_test,b))
newAug,p,swaps = GE_srpp(augA,verbose=True)

permutation vector initialized to:  [0, 1, 2]
Scaling Vector:  [5. 3. 4.]

Pivot Element: 3.0000 

permutation vector:  [0, 1, 2]

 After PASS 1=========: 
 \begin{bmatrix}
  3. & -5. & 1. & 0.\\
  0.33333333 & 3.66666667 & 2.66666667 & 1.\\
  -0.66666667 & -0.33333333 & -3.33333333 & 3.\\
\end{bmatrix}
Scaling Vector:  [5. 3. 4.]

Pivot Element: 3.6667 

permutation vector:  [0, 1, 2]

 After PASS 2=========: 
 \begin{bmatrix}
  3. & -5. & 1. & 0.\\
  0.33333333 & 3.66666667 & 2.66666667 & 1.\\
  -0.66666667 & -0.09090909 & -3.09090909 & 3.09090909\\
\end{bmatrix}


In [ ]:
print("The Permutation Matrix, P:\n",P)
print("Error Norm \n", np.linalg.norm(np.dot(L,U) - np.dot(P,A0)))
print(np.dot(L,U))
print(np.dot(P,A0))

### P LU Factorization 
---
One can use the Gaussian elimination with partial pivoting to decompose the matrix A as follows
$$
P A = L U,\text{ or }\ A = P^T L U;
$$
where $P$ is a permutation matrix, $L$ is a unit lower-triangular matrix and $U$ is an upper triangular matrix.

## An Example From LLM
Solving a linear system using Gaussian elimination with scaled row partial pivoting involves several steps. The goal is to transform the system into an upper triangular form, which can then be solved using back substitution. Scaled row partial pivoting helps to reduce numerical errors by selecting the pivot element with the largest relative magnitude in the column.

### Example System:
Consider the following system of linear equations. <strong>There are som errors in explanation of the algorithm. Find them out and correct them.</strong> 

$$
\begin{cases}
2x + 3y - z = 5 \\
4x + 4y - 3z = 3 \\
2x - 3y + z = -1
\end{cases}
$$

This can be written in matrix form as:

$$
\begin{bmatrix}
2 & 3 & -1 & | & 5 \\
4 & 4 & -3 & | & 3 \\
2 & -3 & 1 & | & -1
\end{bmatrix}
$$

### Step 1: Compute the Scaling Factors
For each row, compute the scaling factor $ s_i$, which is the maximum absolute value in that row (excluding the augmented column).

$$
s_1 = \max{(|2|, |3|, |-1|)} = 3 \\
s_2 = \max{(|4|, |4|, |-3|)} = 4 \\
s_3 = \max{(|2|, |-3|, |1|)} = 3
$$

### Step 2: Select the Pivot Row
For the first column, compute the ratios $|a_{i1}|/s_i$ for each row $ i $:

$$
\text{Ratio for Row 1: } |2| / 3 = 0.6667 \\
\text{Ratio for Row 2: } |4| / 4 = 1.0 \\
\text{Ratio for Row 3: } |2| / 3 = 0.6667
$$

The largest ratio is 1.0, corresponding to Row 2. Therefore, Row 2 is the pivot row for the first step.

### Step 3: Swap Rows (if necessary)
Since Row 2 is already in the second position, no swapping is needed.

### Step 4: Eliminate the First Column Below the Pivot
Use the pivot element \( a_{21} = 4 \) to eliminate the first column elements below it.

- **Row 1:** \( R_1 = R_1 - (a_{11}/a_{21}) \times R_2 = R_1 - (2/4) \times R_2 \)
- **Row 3:** \( R_3 = R_3 - (a_{31}/a_{21}) \times R_2 = R_3 - (2/4) \times R_2 \)

Calculating these:

$$
R_1 = R_1 - 0.5 \times R_2 = [2, 3, -1, 5] - 0.5 \times [4, 4, -3, 3] = [0, 1, 0.5, 3.5] \\
R_3 = R_3 - 0.5 \times R_2 = [2, -3, 1, -1] - 0.5 \times [4, 4, -3, 3] = [0, -5, 2.5, -2.5]
$$

The updated matrix is:

$$
\begin{bmatrix}
0 & 1 & 0.5 & | & 3.5 \\
4 & 4 & -3 & | & 3 \\
0 & -5 & 2.5 & | & -2.5
\end{bmatrix}
$$

### Step 5: Select the Pivot Row for the Second Column
Now, consider the second column below the first pivot row. Compute the ratios $ |a_{i2}| / s_i $ for each remaining row $ i $:

$$
\text{Ratio for Row 1: } |1| / 3 = 0.3333 \\
\text{Ratio for Row 3: } |-5| / 3 = 1.6667
$$

The largest ratio is 1.6667, corresponding to Row 3. Therefore, Row 3 is the pivot row for the second step.

### Step 6: Swap Rows (if necessary)
Swap Row 1 and Row 3:

$$
\begin{bmatrix}
0 & -5 & 2.5 & | & -2.5 \\
4 & 4 & -3 & | & 3 \\
0 & 1 & 0.5 & | & 3.5
\end{bmatrix}
$$

### Step 7: Eliminate the Second Column Below the Pivot
Use the pivot element $ a_{32} = 1$ to eliminate the second column elements below it.

- **Row 1:** $ R_1 = R_1 - (a_{12}/a_{32}) \times R_3 = R_1 - (-5/1) \times R_3 $

Calculating this:

$$
R_1 = R_1 + 5 \times R_3 = [0, -5, 2.5, -2.5] + 5 \times [0, 1, 0.5, 3.5] = [0, 0, 5, 15]
$$

The updated matrix is:

$$
\begin{bmatrix}
0 & 0 & 5 & | & 15 \\
4 & 4 & -3 & | & 3 \\
0 & 1 & 0.5 & | & 3.5
\end{bmatrix}
$$

### Step 8: Back Substitution
Now, the matrix is in upper triangular form. Solve for the variables starting from the bottom:

1. From Row 3: \( 0x + 1y + 0.5z = 3.5 \) → \( y + 0.5z = 3.5 \)
2. From Row 2: \( 4x + 4y - 3z = 3 \)
3. From Row 1: \( 0x + 0y + 5z = 15 \) → \( z = 3 \)

Substitute $ z = 3$ into Row 3:

$$
y + 0.5(3) = 3.5 \rightarrow y = 2
$$

Substitute $ y = 2 $ and $ z = 3 $ into Row 2:

$$
4x + 4(2) - 3(3) = 3 \rightarrow 4x + 8 - 9 = 3 \rightarrow 4x = 4 \rightarrow x = 1
$$

### Final Solution:
$$
x = 1, \quad y = 2, \quad z = 3
$$

This is the solution to the system of linear equations using Gaussian elimination with scaled row partial pivoting.

In [2]:
def bmatrix(A):
    """This function converts a 1-2 dimensional array into a LaTeX B-matrix
    INPUT:a: numpy array
    OUTPUT: LaTeX B-matrix code
    """
    if len(A.shape) > 2:
        raise ValueError('The input array must be a vector or a matrix.')
    
    rows = str(A).replace('[', '').replace(']', '').splitlines()
    ltx_str = [r'\begin{bmatrix}']
    ltx_str += ['  ' + ' & '.join(val.split()) + r'\\' for val in rows]
    ltx_str +=  [r'\end{bmatrix}']
    return '\n'.join(ltx_str)

A = np.array([[12, 5, 2], [20, 4, 8], [ 2, 4, 3], [ 7, 1, 10]])
print (bmatrix(A) + '\n')

B = np.array([[1.2], [3.7], [0.2]])
print (bmatrix(B) + '\n')

C = np.array([1.2, 9.3, 0.6, -2.1])
print (bmatrix(C) + '\n')

\begin{bmatrix}
  12 & 5 & 2\\
  20 & 4 & 8\\
  2 & 4 & 3\\
  7 & 1 & 10\\
\end{bmatrix}

\begin{bmatrix}
  1.2\\
  3.7\\
  0.2\\
\end{bmatrix}

\begin{bmatrix}
  1.2 & 9.3 & 0.6 & -2.1\\
\end{bmatrix}



## Iterative Methods for Linear Systems
---
So far we have only seen some variants of what we call direct linear solvers. These methods have their limitations. For example, for large systems, especially sparse ones, these methods lack scalability. Iterative methods are numerical algorithms to solve linear systems that work far better in several situations. On asking ChatGPT about the benefits of iterative solutions over direct methods such as Gaussian elimination, following are some of the noteworthy arguments in its favor.

Iterative methods for solving linear systems are important for several reasons:

**Scalability**: Iterative methods can handle large and sparse linear systems that may be impractical to solve using direct methods like Gaussian elimination or matrix factorization. This scalability is crucial in fields such as scientific computing and engineering, where systems with millions or even billions of equations are common.

**Memory efficiency**: Direct methods often require storing and manipulating large matrices, which can be memory-intensive. In contrast, iterative methods can be memory-efficient because they do not need to store the entire matrix at once. They work by updating and refining an initial guess for the solution, which can be done iteratively, often requiring less memory.

**Adaptability**: Iterative methods can adapt to the specific characteristics of a problem. They can be designed to exploit known properties of the linear system, such as its sparsity or special structure, leading to faster convergence in many cases.

**Speed**: For certain types of problems, iterative methods can converge to an acceptable solution much faster than direct methods. This speed advantage can be crucial in real-time simulations, optimization problems, and other applications where quick solutions are required.

**Robustness**: Iterative methods can be more robust when dealing with ill-conditioned matrices or systems that are close to singular. They can often continue to make progress toward a solution even when direct methods fail due to numerical instability.

**Parallelism**: Many iterative methods can be parallelized effectively, taking advantage of modern multicore processors and distributed computing environments. This parallelism can lead to significant speedup for solving large linear systems.

**Partial solutions**: In some applications, it may not be necessary to find the exact solution to a linear system. Iterative methods can provide partial solutions that are accurate enough for the specific problem at hand. This can save computational resources.

**Preconditioning**: Iterative methods can be combined with preconditioners, which are techniques that transform the original linear system into one that is easier to solve. This can significantly improve the convergence rate of iterative methods, making them more practical for challenging problems.

**Nonlinear problems**: Iterative methods can be extended to solve nonlinear systems of equations, which are common in optimization, physics simulations, and engineering design. These methods can be used iteratively to approximate solutions to nonlinear problems.

In summary, iterative methods offer versatility, efficiency, and adaptability, making them essential for solving a wide range of linear systems encountered in various scientific, engineering, and computational applications. They are particularly well-suited for handling large, complex, and computationally challenging problems.

Refer to John Foster's page on [iterative methods](https://johnfoster.pge.utexas.edu/numerical-methods-book/LinearAlgebra_IterativeSolvers.html).


### Jacobi Method

To solve the linear system of equations:


\begin{aligned}
2x_1 + x_2 &= 5 \\
x_1 + 3x_2 &= 9
\end{aligned}


We can use the Jacobi iterative method. Start with an initial guess for the solution, e.g., $x_1^{(0)} = 0$ and $x_2^{(0)} = 0$. Then, iterate using the following formulas until convergence is reached:

\begin{aligned}
x_1^{(k+1)} &= \frac{1}{2} \left(5 - x_2^{(k)}\right) \\
x_2^{(k+1)} &= \frac{1}{3} \left(9 - x_1^{(k)}\right)
\end{aligned}


Let's perform a few iterations:


\begin{array}{|c|c|c|c|}
\hline
k & x_1^{(k)} & x_2^{(k)} \\
\hline
0 & 0 & 0 \\
1 & 2.5 & 3 \\
2 & 1.25 & 2.25 \\
3 & 1.375 & 2.125 \\
4 & 1.3125 & 2.1875 \\
\hline
\end{array}


After a few iterations, we observe that the values of $x_1$ and $x_2$ are converging. The solution to the system is approximately $x_1 \approx 1.3125$ and $x_2 \approx 2.1875$.

**Pseudo-code for Jacobi Method**
>  Initialize the iterative solution vector $x^{(0)}$ randomly, or with the zero vector,

>  for k=0:maxIteration, update every element until convergece

>> for i=1:n
$$
x^{(k+1)}_{i}  = \frac{1}{a_{ii}} \left(b_i -\sum_{j\ne i}a_{ij}x^{(k)}_{j}\right).
$$


Here is an example of a linear system in four variables.

\begin{aligned}
4x_1 - x_2 + x_3 + x_4 &= 7 \\
x_1 + 5x_2 - x_3 - x_4 &= 11 \\
x_1 - x_2 + 6x_3 - x_4 &= 13 \\
x_1 - x_2 - x_3 + 7x_4 &= 19
\end{aligned}


We want to solve for the unknown variables $x_1$, $x_2$, $x_3$, and $x_4$ using the Jacobi method. We'll start with an initial guess of $x_1^{(0)} = 0$, $x_2^{(0)} = 0$, $x_3^{(0)} = 0$, and $x_4^{(0)} = 0$.

The Jacobi iteration formula for each component is:


\begin{aligned}
x_1^{(k+1)} &= \frac{1}{4}\left(7 + x_2^{(k)} - x_3^{(k)} - x_4^{(k)}\right) \\
x_2^{(k+1)} &= \frac{1}{5}\left(11 - x_1^{(k)} + x_3^{(k)} + x_4^{(k)}\right) \\
x_3^{(k+1)} &= \frac{1}{6}\left(13 - x_1^{(k)} + x_2^{(k)} + x_4^{(k)}\right) \\
x_4^{(k+1)} &= \frac{1}{7}\left(19 - x_1^{(k)} + x_2^{(k)} + x_3^{(k)}\right)
\end{aligned}


Let's perform a few iterations:


\begin{array}{|c|c|c|c|c|}
\hline
k & x_1^{(k)} & x_2^{(k)} & x_3^{(k)} & x_4^{(k)} \\
\hline
0 & 0 & 0 & 0 & 0 \\
1 & 1.75 & 2.2 & 2.1667 & 2.7143 \\
2 & 2.4429 & 1.8 & 2.7917 & 2.6623 \\
3 & 2.8964 & 1.8679 & 2.9 & 2.6844 \\
4 & 2.8422 & 1.9269 & 2.9556 & 2.6946 \\
\hline
\end{array}


After a few iterations, we observe that the values of $x_1$, $x_2$, $x_3$, and $x_4$ are converging. The solution to the system is approximately $x_1 \approx 2.8422$, $x_2 \approx 1.9269$, $x_3 \approx 2.9556$, and $x_4 \approx 2.6946$.


### Gauss-Seidel Iteration Method

The **Gauss-Seidel** iteration method is similar to Jacobi but it uses the latest iterations of the unknown variables:


So in the previous examples we wil have 

\begin{aligned}
x_1^{(k+1)} &= \frac{1}{4}\left(7 + x_2^{(k)} - x_3^{(k)} - x_4^{(k)}\right) \\
x_2^{(k+1)} &= \frac{1}{5}\left(11 - x_1^{(k+1)} + x_3^{(k)} + x_4^{(k)}\right) \\
x_3^{(k+1)} &= \frac{1}{6}\left(13 - x_1^{(k+1)} + x_2^{(k+1)} + x_4^{(k)}\right) \\
x_4^{(k+1)} &= \frac{1}{7}\left(19 - x_1^{(k+1)} + x_2^{(k+1)} + x_3^{(k+1)}\right)
\end{aligned}

**Pseudo-code for Gauss-Seidel Method**
>  Initialize the iterative solution vector $x^{(0)}$ randomly, or with the zero vector,

>  for k=0:maxIteration, update every element until convergece

>> for i=1:n
$$
x^{(k+1)}_{i}  = \frac{1}{a_{ii}} \left(b_i -\sum_{j< i}a_{ij}x^{(k)}_{j} -\sum_{j> i}a_{ij}x^{(k)}_{j}\right).
$$


In [12]:
# You can modify this code to answer build a function for Gauss-Seidel or SOR.

import numpy as np
'''
Jacobi's iteration method for solving the system of equations Ax=b.
p0 is the initialization for the iteration.
'''
def jacobi(A, b, p0, tol, maxIter=100, verbose=False):
    n=len(A)
    p = p0

    for k in range(maxIter):        
        p_old = p.copy() # In python assignment is not the same as copy
        if verbose:
            print ("\n Iteration {:2d} ".format(k),p_old)
        # Update every component of iterant p
        for i in range(n):
            sumi = b[i];
            for j in range(n):
                if i==j: # Diagonal elements are not included in Jacobi
                    continue;
                sumi = sumi - A[i,j] * p_old[j]
            p[i] = sumi/A[i,i]
                
        my_error = np.linalg.norm(p-p_old)/n
        if verbose:
            print("Relative error in iteration", k+1,":",my_error)
        if my_error<tol:
            print("\n TOLERANCE MET BEFORE MAX-ITERATION.")
            print("Total number of iterations:",k)
            break
    return p;

In [2]:
# Example
A = np.array([[9, -1, 2, 0],
              [-1, 8, -1, -3],
              [2, -1, 6, -1],
              [0, 3, -1, 5]],dtype=float)
b = np.array([5, 13, -7, 12],dtype=float)

In [15]:
np.set_printoptions(precision=12)
x_0 = np.array([1, 0, 6, 0],dtype=float)
soln  = jacobi(A,b,x_0,0.000000001,100, verbose=True)
print("\n The solution is: ",soln)


 Iteration  0  [1. 0. 6. 0.]
Relative error in iteration 1 : 2.2167049565058337

 Iteration  1  [-0.777777777778  2.5            -1.5             3.6           ]
Relative error in iteration 2 : 0.9812869481501886

 Iteration  2  [1.166666666667 2.690277777778 0.109259259259 0.6           ]
Relative error in iteration 3 : 0.34152640021134306

 Iteration  3  [ 0.830195473251  2.009490740741 -1.007175925926  0.807685185185]
Relative error in iteration 4 : 0.06887641390906281

 Iteration  4  [ 1.002649176955  1.90575938786  -0.973869170096  0.99287037037 ]
Relative error in iteration 5 : 0.03171134270600088

 Iteration  5  [ 0.983721969784  2.000923889746 -1.01777809928   1.061770533265]
Relative error in iteration 6 : 0.019700142279340215

 Iteration  6  [ 1.004053343145  2.018906933787 -0.984124919426  0.995890046296]
Relative error in iteration 7 : 0.006056132852133897

 Iteration  7  [ 0.998572974738  2.000949820326 -0.998884951034  0.991830855842]
Relative error in iteration 8 : 0.00